## Extended tests on different areas

This tutorial shows the results of the (i) geometric and (ii) radiometric terrain corrections on the Sentinel-1 GRD product using `sarsen` om 5 zone of the US:
- South of Redmond in Washington
- Grand Canyon in Arizona 
- Near Denver in Colorado
- Clearwater National Forest in Idaho
- Ozark National Forest in Arkansas 

The zone can be selected in the **Area Selection** section

We use a 10-meter resolution DEM, the same resolution of the DEM used to generate the RTC available on the Planetary Computer. The comparison at the end of this notebook demonstrates that the RTC computed by `sarsen` is consistent with the RTC from the Planetary Computer.

Steps:
- Download the Sentinel-1 GRD
- Download the 10-meter DEM
- Compute the GTC using `sarsen`
- Compute the RTC using `sarsen`
- Compare the GTC to the RTC
- Compare the RTC computed using `sarsen` to the RTC already available on the Planetery Computer 

**Note**: Download/retrieval steps are slower on local machines compared to the Planetary Computer. In future versions, it will be possible to access data via [fsspec](https://filesystem-spec.readthedocs.io/en/latest/) without having to download data locally.

<hr style="border:2px solid blue"> </hr>

### Import

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (10, 7)

In [ ]:
import os
import tempfile

# enable the `.rio` accessor
import numpy as np
import rioxarray  # noqa: F401
import xarray as xr

from sarsen import apps

In [ ]:
import adlfs
import planetary_computer
import pystac_client
import stackstac

### Area Selection

#### Availables Areas

In [ ]:
processing_definitions = dict(
    washington_south_of_redmond=dict(
        lon=-121.95,
        lat=47.04,
        bbox=[-121.95 - 0.2, 47.04 - 0.2, -121.95 + 0.2, 47.04 + 0.2],
        product_folder="GRD/2021/12/17/IW/DV/S1B_IW_GRDH_1SDV_20211217T141304_20211217T141329_030066_039705_9048",
    ),
    arizona_grand_canyon=dict(
        lon=-113.32,
        lat=36.11,
        product_folder="GRD/2021/12/10/IW/DV/S1A_IW_GRDH_1SDV_20211210T133532_20211210T133557_040947_04DCF6_4F80",
    ),
    colorado_denver=dict(
        lon=-106.28,
        lat=39.65,
        product_folder="GRD/2022/1/12/IW/DV/S1A_IW_GRDH_1SDV_20220112T130955_20220112T131020_041428_04ED13_2B27",
    ),
    idaho_clearwater_national_forest=dict(
        lon=-115.05,
        lat=46.04,
        product_folder="GRD/2021/12/2/IW/DV/S1B_IW_GRDH_1SDV_20211202T134837_20211202T134902_029847_039022_4099",
    ),
    arkansas_ozark_national_forest=dict(
        lon=-93.14367475,
        lat=35.70534225,
        product_folder="GRD/2021/12/8/IW/DV/S1A_IW_GRDH_1SDV_20211208T001209_20211208T001234_040910_04DBA7_F03C",
    ),
)

**Area selection**

Select a zone from the following list:
- "washington_south_of_redmond"
- "arizona_grand_canyon"
- "colorado_denver"
- "idaho_clearwater_national_forest"
- "arkansas_ozark_national_forest"

In [ ]:
zone = "colorado_denver"

#### processing definition

In [ ]:
# Area of interest definition
lon = processing_definitions[zone]["lon"]
lat = processing_definitions[zone]["lat"]
bbox = [lon - 0.2, lat - 0.2, lon + 0.2, lat + 0.2]

# Product Definition
product_folder = processing_definitions[zone]["product_folder"]
measurement_group = "IW/VV"

# create a temporary directory where to store downloaded data
tmp_dir = tempfile.gettempdir()
# DEM path
dem_path = os.path.join(tmp_dir, "dem.tif")

tmp_dir

### DEMs discovery

Here we use the DEM with a 10-meter ground sample distance (GDS) available on the Planetary Computer. Note that **any DEM supported by GDAL/Proj can be used**.

Using `pystac_client` we can search the Planetary Computer's STAC endpoint for items matching our query parameters.  
As multiple DEMs acquired at different times are available in this area, we select the DEMs with 10-meter GDS and perform the average of the remaining DEMs along the time dimension.

In [ ]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1"
)
search = catalog.search(collections="3dep-seamless", bbox=bbox)
items = list(search.get_items())

Here we load the data into an xarray `DataArray` using stackstac.

In [ ]:
# select DEMs with resolution 10 meters
items_high_res = [
    planetary_computer.sign(item).to_dict()
    for item in items
    if item.properties["gsd"] == 10
]

dem_raster_all = stackstac.stack(items_high_res, bounds=bbox).squeeze()
dem_raster_all

#### DEMs average along the time dimension

In [ ]:
dem_raster_geo = dem_raster_all.compute()
if "time" in dem_raster_geo.dims:
    dem_raster_geo = dem_raster_geo.mean("time")
_ = dem_raster_geo.rio.set_crs(dem_raster_all.rio.crs)

#### Convert the DEM in UTM coordinates

In order to facilitate the comparison between the RTC computed by sarsen with the RTC available on the Planetery Computer, here we convert the DEM in UTM.

In [ ]:
dem_raster_geo

In [ ]:
# find the UTM zone and project in UTM
t_srs = dem_raster_geo.rio.estimate_utm_crs()
dem_raster = dem_raster_geo.rio.reproject(t_srs, resolution=(10, 10))
dem_raster

In [ ]:
# crop DEM to our area of interest and save it
# dem_corners = dict(x=slice(565000, 594000), y=slice(5220000, 5190000))
# dem_raster_crop = dem_raster.sel(**dem_corners)

dem_corners = dict(
    x=slice(len(dem_raster.x) // 2 - 1000, len(dem_raster.x) // 2 + 1000),
    y=slice(len(dem_raster.y) // 2 - 1000, len(dem_raster.y) // 2 + 1000),
)
print(dem_corners)
dem_raster_crop = dem_raster.isel(**dem_corners)

# dem_raster_crop = xr.where(dem_raster_crop >= 1.7976931348623157e+308, np.nan, dem_raster_crop)

dem_raster_crop.rio.to_raster(dem_path)
dem_raster_crop

In [ ]:
dem_raster_crop.plot()
_ = plt.title("DEM in UTM coordinates")

### Retrieve Sentinel-1 GRD

#### Define GRD parameters

In [ ]:
grd_account_name = "sentinel1euwest"
grd_storage_container = "s1-grd"
grd_product_folder = f"{grd_storage_container}/{product_folder}"
grd_local_path = os.path.join(tmp_dir, product_folder)

#### Retrieve Sentinel-1 GRD

In [ ]:
grd_token = planetary_computer.sas.get_token(
    grd_account_name, grd_storage_container
).token
grd_fs = adlfs.AzureBlobFileSystem(grd_account_name, credential=grd_token)
grd_fs.ls(f"{grd_product_folder}/manifest.safe")

In [ ]:
grd_fs.get(grd_product_folder, grd_local_path, recursive=True)
!ls -d {grd_local_path}

### Processing

#### GTC

Here we compute the geometric terrain correction.

Input parameters:
- `product_urlpath`: product path
- `measurement_group`: band to be processed in the form {swath}/{polarization} (see [xarray-sentinel](https://pypi.org/project/xarray-sentinel/) for more details)
- `dem_urlpath`: path to the input DEM. sarsen supports all DEMs supported by GDAL/Proj for ECEF-translation. 
- `interp_method`: interpolation method, sarsen supports all interpolation methods supported by [xarray.Dataset.interp](https://xarray.pydata.org/en/stable/generated/xarray.Dataset.interp.html)
- `chunks`: dask chunks
- `output_urlpath`: output path

The output is the input SAR image resampled on DEM coordinates. 

In [ ]:
gtc = apps.terrain_correction(
    product_urlpath=grd_local_path,
    measurement_group=measurement_group,
    dem_urlpath=dem_path,
    output_urlpath=os.path.join(
        tmp_dir, os.path.basename(product_folder) + ".10m.GTC.tif"
    ),
)

In [ ]:
_ = gtc.plot(vmax=0.4)

#### RTC
`sarsen` implements the radiometric terrain-correction [Gamma Flattening](https://ieeexplore.ieee.org/document/5752845) algorithm.

#### Input parameters
- `correct_radiometry`: `correct_radiometry`: default `None`. If `correct_radiometry=None`the radiometric terrain correction is not applied. `correct_radiometry=gamma_bilinear` applies the gamma flattening classic algorithm using bilinear interpolation to compute the weights. `correct_radiometry=gamma_nearest` applies the gamma flattening using nearest neighbours instead of bilinear interpolation. 'gamma_nearest' significantly reduces the processing time.
- `grouping_area_factor`: scaling factor for the size of the image pixel where the areas are summed. By default, the `grouping_area_factor` is `(1, 1)`, which corresponds to Sentinel-1 input product pixel size. The `grouping_area_factor` shall be increased if the DEM resolution is lower than the Sentinel-1 input product resolution to avoid gaps and distortions the normalization factor. It can be also used to to speed up the computation or  the DEM resolution is lower than the Sentinel-1 input product resolution.  


**Note**: The `grouping_area_factor` can be increased (i) to speed up the processing or (ii) when the input DEM resolution is low. The Gamma Flattening usually works properly if the pixel size of the input DEM is much smaller than the pixel size of the input Sentinel-1 product. Otherwise, the output may have radiometric distortions. This problem can be avoided by increasing the `grouping_area_factor`. Be aware that `grouping_area_factor` too high may degrade the final result.

**Note:** As the RTC genaration step loads data into the memory, it may take serveral minutes (about 10 minutes on the Planetary Computer). The performances will be improved in the next releases of `sarsen`.

In [ ]:
rtc = apps.terrain_correction(
    grd_local_path,
    measurement_group=measurement_group,
    dem_urlpath=dem_path,
    correct_radiometry="gamma_bilinear",
    output_urlpath=os.path.join(
        tmp_dir, os.path.basename(product_folder) + ".10m.RTC.tif"
    ),
    grouping_area_factor=(3, 3),
)

### Comparison between GTC and RTC

In [ ]:
f, axes = plt.subplots(nrows=1, ncols=2, figsize=(30, 12))

gtc.plot(ax=axes[0], vmax=0.4)
axes[0].grid(c="red")
plt.title("GTC")

rtc.plot(ax=axes[1], vmax=0.4)
axes[1].grid(c="red")
plt.title("RTC")

plt.tight_layout()

### Comparison between sarsen RTC and Planetary Computer RTC


#### Define RTC parameters

In [ ]:
rtc_account_name = "sentinel1euwestrtc"
rtc_storage_container = "sentinel1-grd-rtc"
rtc_product_folder = f"{rtc_storage_container}/{product_folder}"
rtc_local_path = os.path.join(tmp_dir, rtc_product_folder)

#### Retrieve Sentinel-1 RTC

In [ ]:
rtc_token = planetary_computer.sas.get_token(
    rtc_account_name, rtc_storage_container
).token
rtc_fs = adlfs.AzureBlobFileSystem(rtc_account_name, credential=rtc_token)
rtc_fs.ls(rtc_product_folder)

In [ ]:
rtc_fs.get(f"{rtc_product_folder}", rtc_local_path, recursive=True)
!ls -d {rtc_local_path}

#### Plot sarsen RTC and Planetary Computer RTC

In [ ]:
rtc_pc = xr.open_dataarray(
    rtc_local_path + "/measurement/iw-vv.rtc.tiff", cache=False
).drop("band")

dem_corners_xy = {
    "x": slice(dem_raster_crop.x[0], dem_raster_crop.x[-1]),
    "y": slice(dem_raster_crop.y[0], dem_raster_crop.y[-1]),
}

rtc_pc = rtc_pc.sel(dem_corners_xy)
rtc_pc

In [ ]:
f, axes = plt.subplots(ncols=2, figsize=(30, 12))

rtc_pc.plot(ax=axes[0], vmax=0.4)
axes[0].set_title("Planetary Computer RTC")
axes[0].grid(c="red")

rtc.plot(ax=axes[1], vmax=0.4)
axes[1].set_title("sarsen RTC")
axes[1].grid(c="red")

plt.tight_layout()